In [ ]:
# Imports
import os
import numpy as np
import pandas as pd
import nibabel as nb
import warnings
warnings.filterwarnings('ignore')

# Figure imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.express as px
from plot_utils import plotly_template

# Define parameters
subjects = ['sub-001', 'sub-002', 'sub-003', 'sub-004',
            'sub-005', 'sub-006', 'sub-007', 'sub-008']
subjects_plot = ['sub-001', 'sub-002', 'sub-003', 'sub-004',
                 'sub-005', 'sub-006', 'sub-007', 'sub-008', 'group']
tasks = ['FullScreen', 'FullScreenAttendFix', 'FullScreenAttendBar']
rois = ['V1', 'V2', 'V3', 'V3AB', 'hMT+', 'LO',
        'VO', 'iIPS', 'sIPS', 'iPCS', 'sPCS', 'mPCS']

# Define folders
base_dir = '/home/mszinte/disks/meso_S/data/gaze_prf'
bids_dir = "{}".format(base_dir)
pp_dir = "{}/derivatives/pp_data".format(base_dir)

# Threshold values
amp_th = 0
r2_th = [0, 1]
ecc_th = [0.1, 20]
sd_th = [0.1, 20]
cortical_mask = 'cortical'
best_voxels_num = 250
# type_analysis =''
type_analysis = '_best{}'.format(best_voxels_num)


In [ ]:
# Create pRF threshold masks
for task in tasks:
    for subject in subjects:
    
        # define folders
        fit_dir = '{}/{}/prf/fit'.format(pp_dir, subject)
        mask_dir = '{}/{}/masks'.format(pp_dir, subject)
        
        # Load fit parameters
        r2_im = nb.load('{}/{}_task-{}_par-r2.nii.gz'.format(fit_dir,subject,task))
        ecc_im = nb.load('{}/{}_task-{}_par-ecc.nii.gz'.format(fit_dir,subject,task))
        sd_im = nb.load('{}/{}_task-{}_par-sd.nii.gz'.format(fit_dir,subject,task))
        amp_im = nb.load('{}/{}_task-{}_par-amplitude.nii.gz'.format(fit_dir,subject,task))

        # Create param matrix
        r2_idx, ecc_idx, sd_idx, amp_idx, mask_idx = 0, 1, 2, 3, 4
        param_mat = np.zeros((r2_im.shape[0],r2_im.shape[1],r2_im.shape[2],5))*np.nan
        param_mat[...,r2_idx] = r2_im.get_fdata()
        param_mat[...,ecc_idx] = ecc_im.get_fdata()
        param_mat[...,sd_idx] = sd_im.get_fdata()
        param_mat[...,amp_idx] = amp_im.get_fdata()
 
        # Threshold param matrix by masking R2
        param_mat_th = param_mat
        amp_down =  param_mat_th[...,amp_idx] > amp_th
        r2_th_down, r2_th_up = param_mat_th[...,r2_idx] >= r2_th[0],  param_mat_th[...,r2_idx] <= r2_th[1]
        ecc_th_down, ecc_th_up = param_mat_th[...,ecc_idx] >= ecc_th[0], param_mat_th[...,ecc_idx] <= ecc_th[1]
        sd_th_down, sd_th_up = param_mat_th[...,sd_idx] >= sd_th[0], param_mat_th[...,sd_idx] <= sd_th[1]
        all_th = np.array((amp_down, r2_th_down, r2_th_up, ecc_th_down, ecc_th_up, sd_th_down, sd_th_up)) 
        
        # create threhsold mask
        param_mat[np.logical_and.reduce(all_th)==False, mask_idx] = np.nan
        param_mat[np.logical_and.reduce(all_th)==True, mask_idx] = 1
        
        # save threshold masks
        th_mask_file = "{}/{}_task-{}_prf_threshold.nii.gz".format(mask_dir, subject, task)
        print('saving {}'.format(th_mask_file))
        th_mask_img = nb.Nifti1Image(dataobj=param_mat[...,mask_idx], affine=r2_im.affine, header=r2_im.header)
        th_mask_img.to_filename(th_mask_file)
        
    

In [ ]:
# Create TSV files
for task in tasks:
    for subject_num, subject in enumerate(subjects):
        # define folders
        fit_dir = '{}/{}/prf/fit'.format(pp_dir, subject)
        mask_dir = '{}/{}/masks'.format(pp_dir, subject)
        tsv_dir = '{}/{}/prf/tsv'.format(pp_dir, subject)
        group_tsv_dir = '{}/{}/prf/tsv'.format(pp_dir, 'group')
        try: os.makedirs(tsv_dir)
        except: pass
        try: os.makedirs(group_tsv_dir)
        except: pass

        # load pRF threshold masks
        th_mat = nb.load('{}/{}_task-{}_prf_threshold.nii.gz'.format(mask_dir,subject,task)).get_fdata()

        # load fit parameters x by threshold
        r2_th_mat = nb.load('{}/{}_task-{}_par-r2.nii.gz'.format(fit_dir,subject,task)).get_fdata()*th_mat
        ecc_th_mat = nb.load('{}/{}_task-{}_par-ecc.nii.gz'.format(fit_dir,subject,task)).get_fdata()*th_mat
        sd_th_mat = nb.load('{}/{}_task-{}_par-sd.nii.gz'.format(fit_dir,subject,task)).get_fdata()*th_mat
        x_th_mat = nb.load('{}/{}_task-{}_par-x.nii.gz'.format(fit_dir,subject,task)).get_fdata()*th_mat
        y_th_mat = nb.load('{}/{}_task-{}_par-y.nii.gz'.format(fit_dir,subject,task)).get_fdata()*th_mat
        amp_th_mat = nb.load('{}/{}_task-{}_par-amplitude.nii.gz'.format(fit_dir,subject,task)).get_fdata()*th_mat
        bsl_th_mat = nb.load('{}/{}_task-{}_par-baseline.nii.gz'.format(fit_dir,subject,task)).get_fdata()*th_mat
        
        # creat tsv
        for roi_num, roi in enumerate(rois):
            # load roi
            lh_mat = nb.load("{}/{}_{}_L.nii.gz".format(mask_dir, roi, cortical_mask)).get_fdata()
            rh_mat = nb.load("{}/{}_{}_R.nii.gz".format(mask_dir, roi, cortical_mask)).get_fdata()
            roi_mat = lh_mat + rh_mat
            roi_mat[roi_mat==0] = np.nan

            # select data by roi mask
            r2_roi_th_mat = r2_th_mat[roi_mat==True]
            ecc_roi_th_mat = ecc_th_mat[roi_mat==True]
            sd_roi_th_mat = sd_th_mat[roi_mat==True]
            x_roi_th_mat = x_th_mat[roi_mat==True]
            y_roi_th_mat = y_th_mat[roi_mat==True]
            amp_roi_th_mat = amp_th_mat[roi_mat==True]
            bsl_roi_th_mat = bsl_th_mat[roi_mat==True]
            
            # create dataframe
            task_roi_mat = [roi] * r2_roi_th_mat.shape[0]
            subject_roi_mat = [subject] * r2_roi_th_mat.shape[0]

            df_roi = pd.DataFrame({'subject': subject_roi_mat,
                                   'roi': task_roi_mat,
                                   'r2': r2_roi_th_mat,
                                   'ecc': ecc_roi_th_mat,
                                   'sd': sd_roi_th_mat,
                                   'x': x_roi_th_mat,
                                   'y': y_roi_th_mat,
                                   'amplitude': amp_roi_th_mat,
                                   'baseline': bsl_roi_th_mat,
                                  })
            
            # rank based on r2
            if task == 'FullScreen': 
                df_roi['rank_fs_r2']=df_roi.groupby('roi')['r2'].rank(method='max',ascending=False)
            else:
                df_fs = pd.read_csv("{}/{}_task-FullScreen_prf_threshold_par.tsv".format(tsv_dir,subject,task),sep="\t")
                df_roi['rank_fs_r2'] = np.array(df_fs.loc[(df_fs.roi == roi)]['rank_fs_r2'])
                
            # get best voxels
            df_best_roi = df_roi[(df_roi.rank_fs_r2<best_voxels_num)]
            
            # across roi
            if roi_num > 0: 
                df = pd.concat([df,df_roi], ignore_index=True)
                df_best = pd.concat([df_best,df_best_roi], ignore_index=True)                
            else:
                df = df_roi
                df_best = df_best_roi
        
        # save dataframe
        df_fn = "{}/{}_task-{}_prf_threshold_par.tsv".format(tsv_dir,subject,task)
        print('saving {}'.format(df_fn))
        df.to_csv(df_fn, sep="\t", na_rep='NaN',index=False)
        
        df_best_fn = "{}/{}_task-{}_prf_threshold_par_best{}.tsv".format(tsv_dir,subject,task,int(best_voxels_num))
        print('saving {}'.format(df_best_fn))
        df_best.to_csv(df_best_fn, sep="\t", na_rep='NaN',index=False)
        
        # across subject
        if subject_num == 0: df_group = df
        else: df_group = pd.concat([df_group, df])
        
        if subject_num == 0: df_best_group = df_best
        else: df_best_group = pd.concat([df_best_group, df_best])
        
    # save group data
    df_group_fn = "{}/group_task-{}_prf_threshold_par.tsv".format(group_tsv_dir,task)
    print('saving {}'.format(df_group_fn))
    df_group.to_csv(df_group_fn, sep="\t", na_rep='NaN')
    
    df_best_group_fn = "{}/group_task-{}_prf_threshold_par_best{}.tsv".format(group_tsv_dir,task,int(best_voxels_num))
    print('saving {}'.format(df_best_group_fn))
    df_best_group.to_csv(df_best_group_fn, sep="\t", na_rep='NaN')

In [ ]:
# General figure settings
template_specs = dict(  axes_color="rgba(0, 0, 0, 1)",
                        axes_width=2,
                        axes_font_size=13,
                        bg_col="rgba(255, 255, 255, 1)",
                        font='Arial',
                        title_font_size=15,
                        plot_width=1.5)
fig_template = plotly_template(template_specs)

y_range_stair = [0,15]
y_range_perf = [0,1]
ab_line_col = "rgba(0, 0, 0, 1)"
af_line_col = "rgba(157, 157, 157, 1)"
ab_area_col = "rgba(0, 0, 0, 0.3)"
af_area_col = "rgba(157, 157, 157, 0.3)"

# Subplot settings
rows, cols = 1, 1
fig_height, fig_width = 800,400
column_widths,row_heights = [1],[1]
sb_specs = [[{}]]
hovertemplate_task = 'ROI: %{y}<br>' + 'R2: %{x:1.2f}'

if type_analysis == '_best{}'.format(best_voxels_num):
    xaxis_range = [0,0.8]
    xaxis_dtick = 0.2
else:
    xaxis_range = [0,0.4]
    xaxis_dtick = 0.1

for subject in subjects_plot:
    subplot_titles = ['<b>Attention effect</b> ({})'.format(subject)]

    fig = make_subplots(rows=rows, cols=cols, specs=sb_specs, print_grid=False, vertical_spacing=0.15, horizontal_spacing=0.05, 
                    column_widths=column_widths, row_heights=row_heights,  subplot_titles=subplot_titles)
    
    # Get Data
    tsv_dir = '{}/{}/prf/tsv'.format(pp_dir, subject)
    af_df = pd.read_csv("{}/{}_task-FullScreenAttendFix_prf_threshold_par{}.tsv".format(tsv_dir,subject,type_analysis), sep="\t")
    ab_df = pd.read_csv("{}/{}_task-FullScreenAttendBar_prf_threshold_par{}.tsv".format(tsv_dir,subject,type_analysis), sep="\t")
    
    x_af, x_ab = af_df.groupby('roi').r2.mean(), ab_df.groupby('roi').r2.mean()
    if subject == 'group': 
        x_af_eb = af_df.groupby(['roi','subject']).r2.mean().reset_index(name='r2').groupby('roi').r2.std()/np.sqrt(len(subjects)-1)
        x_ab_eb = ab_df.groupby(['roi','subject']).r2.mean().reset_index(name='r2').groupby('roi').r2.std()/np.sqrt(len(subjects)-1)
    else: 
        x_af_eb, x_ab_eb = af_df.groupby('roi').r2.std(), ab_df.groupby('roi').r2.std()
    
    # bar plot
    fig.append_trace(go.Bar(y=rois, x=x_ab, orientation='h',
                            showlegend=True, legendgroup='attend-bar', name='<i>attend-bar<i>',
                            marker_color=ab_line_col, hovertemplate=hovertemplate_task,
                            error_x_type='data', error_x_symmetric=False, error_x_array=x_ab_eb, error_x_arrayminus=x_ab_eb),row=1, col=1)
    
    fig.append_trace(go.Bar(y=rois, x=x_af, orientation='h',
                            showlegend=True, legendgroup='attend-fix', name='<i>attend-fix<i>', 
                            marker_color=af_line_col, hovertemplate=hovertemplate_task,
                            error_x_type='data', error_x_symmetric=False, error_x_array=x_af_eb, error_x_arrayminus=x_af_eb),row=1, col=1)
    # set axis
    for row in np.arange(rows):
        for col in np.arange(cols):
            fig.update_xaxes(visible=True, ticklen=8, linewidth=template_specs['axes_width'], row=row+1, col=col+1)
            fig.update_yaxes(visible=True, ticklen=8, linewidth=template_specs['axes_width'], row=row+1, col=col+1, autorange='reversed')

    # set figure
    fig.layout.update(xaxis_range=xaxis_range, xaxis_title='R2', xaxis_dtick=xaxis_dtick,
                      yaxis_title='',bargap=0.30,
                      template=fig_template, width=fig_width, height=fig_height, margin_l=50, margin_r=20, margin_t=50, margin_b=70,
                      legend_yanchor='top', 
                      legend_y=0.995, legend_xanchor='left', legend_x=0.9, legend_bgcolor='rgba(255,255,255,0)', legend_tracegroupgap=1)
    
    # show and save figure
    fig.show(config={"displayModeBar": False})
    fig.write_image("{}/{}_attcmp{}.pdf".format(tsv_dir, subject, type_analysis))
    fig.write_html("{}/{}_attcmp{}.html".format(tsv_dir, subject, type_analysis),config={"displayModeBar": False})

#### Eccentricity effect

In [ ]:
# key parameter
n_ecc_bins=5

# General figure settings
template_specs = dict(  axes_color="rgba(0, 0, 0, 1)",
                        axes_width=2,
                        axes_font_size=13,
                        bg_col="rgba(255, 255, 255, 1)",
                        font='Arial',
                        title_font_size=15,
                        plot_width=1.5)
fig_template = plotly_template(template_specs)

y_range_stair = [0,15]
y_range_perf = [0,1]
ab_line_col = "rgba(0, 0, 0, 1)"
af_line_col = "rgba(157, 157, 157, 1)"
ab_area_col = "rgba(0, 0, 0, 0.3)"
af_area_col = "rgba(157, 157, 157, 0.3)"

# Subplot settings
rows, cols = 3, 4
fig_height, fig_width = 1000,1000
column_widths,row_heights = [1,1,1,1],[1,1,1]
sb_specs = [[{},{},{},{}],[{},{},{},{}],[{},{},{},{}]]
hovertemplate_task = 'R2: %{y:1.2f}<br>' + 'Ecc.: %{x:1.2f} dva'
xaxis_range = [0,20]
yaxis_range = [0,0.8]
                                 
if type_analysis == '_best{}'.format(best_voxels_num):
    yaxis_range = [0,0.8]
    yaxis_dtick = 0.2
else: 
    yaxis_range = [0,0.4]
    yaxis_dtick = 0.1

for subject_num, subject in enumerate(subjects_plot):

    subplot_titles = ['<b>V1</b> ({})'.format(subject), '<b>V2</b> ({})'.format(subject), '<b>V3</b> ({})'.format(subject), '<b>V3AB</b> ({})'.format(subject),
                  '<b>hMT+</b> ({})'.format(subject), '<b>VO</b> ({})'.format(subject), '<b>LO</b> ({})'.format(subject), '<b>iIPS</b> ({})'.format(subject),
                  '<b>sIPS</b> ({})'.format(subject), '<b>iPCS</b> ({})'.format(subject), '<b>sPCS</b> ({})'.format(subject), '<b>mPCS</b> ({})'.format(subject)]

    fig = make_subplots(rows=rows, cols=cols, specs=sb_specs, print_grid=False, vertical_spacing=0.15, horizontal_spacing=0.05, 
                        column_widths=column_widths, row_heights=row_heights, subplot_titles=subplot_titles)

    # Get Data
    tsv_dir = '{}/{}/prf/tsv'.format(pp_dir, subject)
    af_df = pd.read_csv("{}/{}_task-FullScreenAttendFix_prf_threshold_par{}.tsv".format(tsv_dir,subject,type_analysis), sep="\t")
    ab_df = pd.read_csv("{}/{}_task-FullScreenAttendBar_prf_threshold_par{}.tsv".format(tsv_dir,subject,type_analysis), sep="\t")

    rows, cols = 3, 4
    for roi_num, roi in enumerate(rois):

        # attention-bar bin
        ab_df_binned = ab_df[ab_df.roi==roi].assign(ecc_bin=pd.qcut(ab_df.ecc, n_ecc_bins))
        ab_df_bin_roi = pd.DataFrame({'roi':[roi] * n_ecc_bins,
                                      'ecc_bin_num':np.arange(0,n_ecc_bins),
                                      'r2_mean':np.array(ab_df_binned.groupby('ecc_bin').r2.mean()),
                                      'r2_std':np.array(ab_df_binned.groupby('ecc_bin').r2.std()),
                                      'ecc_mean':np.array(ab_df_binned.groupby('ecc_bin').ecc.mean()),
                                      'ecc_std':np.array(ab_df_binned.groupby('ecc_bin').ecc.std())})
        if roi_num == 0: ab_df_bin = ab_df_bin_roi
        else: ab_df_bin = pd.concat([ab_df_bin,ab_df_bin_roi], ignore_index=True)

        # attention-fix bins
        af_df_binned = af_df[af_df.roi==roi].assign(ecc_bin=pd.qcut(af_df.ecc, n_ecc_bins))
        af_df_bin_roi = pd.DataFrame({'roi':[roi] * n_ecc_bins,
                                      'ecc_bin_num':np.arange(0,n_ecc_bins),
                                      'r2_mean':np.array(af_df_binned.groupby('ecc_bin').r2.mean()),
                                      'r2_std':np.array(af_df_binned.groupby('ecc_bin').r2.std()),
                                      'ecc_mean':np.array(af_df_binned.groupby('ecc_bin').ecc.mean()),
                                      'ecc_std':np.array(af_df_binned.groupby('ecc_bin').ecc.std())})
        if roi_num == 0: af_df_bin = af_df_bin_roi
        else: af_df_bin = pd.concat([af_df_bin,af_df_bin_roi], ignore_index=True)

        # plot data
        row_num = np.ceil((roi_num+1)/cols)
        if roi_num == 0 or roi_num == 4 or roi_num == 8: col_num = 1
        elif roi_num == 1 or  roi_num == 5 or roi_num == 9: col_num = 2
        elif roi_num == 2 or  roi_num == 6 or roi_num == 10: col_num = 3
        elif roi_num == 3 or  roi_num == 7 or roi_num == 11: col_num = 4

        if roi_num == 0 :showlegend=True
        else: showlegend=False

        # scatter plot            
        fig.append_trace(go.Scatter(x=ab_df_bin[ab_df_bin.roi==roi].ecc_mean,
                                    y=ab_df_bin[ab_df_bin.roi==roi].r2_mean,
                                    # error_x_array=ab_df_bin[ab_df_bin.roi==roi].ecc_std, 
                                    # error_x_arrayminus=ab_df_bin[ab_df_bin.roi==roi].ecc_std, 
                                    # error_y_array=ab_df_bin[ab_df_bin.roi==roi].r2_std, 
                                    # error_y_arrayminus=ab_df_bin[ab_df_bin.roi==roi].r2_std,
                                    # error_x_type='data', error_y_type='data', error_x_symmetric=False, error_y_symmetric=False, 
                                    legendgroup='attend-bar', name='<i>attend-bar<i>',
                                    showlegend=showlegend, mode='markers+lines', marker_color=ab_line_col, hovertemplate=hovertemplate_task,
                                    line_width=1, opacity=1, marker_size= 8),row=int(row_num), col=int(col_num))

        fig.append_trace(go.Scatter(x=af_df_bin[af_df_bin.roi==roi].ecc_mean, 
                                    y=af_df_bin[af_df_bin.roi==roi].r2_mean,
                                    # error_x_array=af_df_bin[af_df_bin.roi==roi].ecc_std, 
                                    # error_x_arrayminus=af_df_bin[af_df_bin.roi==roi].ecc_std, 
                                    # error_y_array=af_df_bin[af_df_bin.roi==roi].r2_std, 
                                    # error_y_arrayminus=af_df_bin[af_df_bin.roi==roi].r2_std,
                                    # error_x_type='data', error_y_type='data', error_x_symmetric=False, error_y_symmetric=False, 
                                    legendgroup='attend-fix', name='<i>attend-fix<i>',
                                    showlegend=showlegend, mode='markers+lines', marker_color=af_line_col, hovertemplate=hovertemplate_task,
                                    line_width=1, opacity=1, marker_size= 8),row=int(row_num), col=int(col_num))

    if subject != 'group':
        if subject_num == 0:  ab_df_group, af_df_group = ab_df_bin, af_df_bin
        else: ab_df_group, af_df_group = pd.concat([ab_df_group, ab_df_bin]), pd.concat([af_df_group, af_df_bin])

    # set axis
    for row in np.arange(rows):
        for col in np.arange(cols):
            fig.update_xaxes(visible=True, ticklen=8, linewidth=template_specs['axes_width'], row=row+1, col=col+1)
            fig.update_yaxes(visible=True, ticklen=8, linewidth=template_specs['axes_width'], row=row+1, col=col+1)

    # set figure 
    fig.layout.update(yaxis_range=yaxis_range,   yaxis_title='R2',   yaxis_dtick=yaxis_dtick,   yaxis2_range=yaxis_range,  yaxis2_title='R2',  yaxis2_dtick=yaxis_dtick,
                      yaxis3_range=yaxis_range,  yaxis3_title='R2',  yaxis3_dtick=yaxis_dtick,  yaxis4_range=yaxis_range,  yaxis4_title='R2',  yaxis4_dtick=yaxis_dtick,
                      yaxis5_range=yaxis_range,  yaxis5_title='R2',  yaxis5_dtick=yaxis_dtick,  yaxis6_range=yaxis_range,  yaxis6_title='R2',  yaxis6_dtick=yaxis_dtick,
                      yaxis7_range=yaxis_range,  yaxis7_title='R2',  yaxis7_dtick=yaxis_dtick,  yaxis8_range=yaxis_range,  yaxis8_title='R2',  yaxis8_dtick=yaxis_dtick,
                      yaxis9_range=yaxis_range,  yaxis9_title='R2',  yaxis9_dtick=yaxis_dtick,  yaxis10_range=yaxis_range, yaxis10_title='R2', yaxis10_dtick=yaxis_dtick,
                      yaxis11_range=yaxis_range, yaxis11_title='R2', yaxis11_dtick=yaxis_dtick, yaxis12_range=yaxis_range, yaxis12_title='R2', yaxis12_dtick=yaxis_dtick,

                      xaxis_range=xaxis_range,   xaxis_title='Eccentricity (dva)',   xaxis_dtick=5,   xaxis2_range=xaxis_range,  xaxis2_title='Eccentricity (dva)',  xaxis2_dtick=5,
                      xaxis3_range=xaxis_range,  xaxis3_title='Eccentricity (dva)',  xaxis3_dtick=5,  xaxis4_range=xaxis_range,  xaxis4_title='Eccentricity (dva)',  xaxis4_dtick=5,
                      xaxis5_range=xaxis_range,  xaxis5_title='Eccentricity (dva)',  xaxis5_dtick=5,  xaxis6_range=xaxis_range,  xaxis6_title='Eccentricity (dva)',  xaxis6_dtick=5,
                      xaxis7_range=xaxis_range,  xaxis7_title='Eccentricity (dva)',  xaxis7_dtick=5,  xaxis8_range=xaxis_range,  xaxis8_title='Eccentricity (dva)',  xaxis8_dtick=5,
                      xaxis9_range=xaxis_range,  xaxis9_title='Eccentricity (dva)',  xaxis9_dtick=5,  xaxis10_range=xaxis_range, xaxis10_title='Eccentricity (dva)', xaxis10_dtick=5,
                      xaxis11_range=xaxis_range, xaxis11_title='Eccentricity (dva)', xaxis11_dtick=5, xaxis12_range=xaxis_range, xaxis12_title='Eccentricity (dva)', xaxis12_dtick=5, 

                      template=fig_template, width=fig_width, height=fig_height, margin_l=50, margin_r=20, margin_t=50, margin_b=70,
                      legend_yanchor='top', legend_y=0.99, legend_xanchor='left', legend_x=0.1, legend_bgcolor='rgba(255,255,255,0)', legend_tracegroupgap=0.05)

    # show and save figure
    fig.show(config={"displayModeBar": False})
    fig.write_image("{}/{}_attcmp_r2_ecc{}.pdf".format(tsv_dir, subject, type_analysis))
    fig.write_html("{}/{}_attcmp_r2_ecc{}.html".format(tsv_dir, subject, type_analysis),config={"displayModeBar": False})